#**스마트폰 센서 데이터 기반 모션 분류**
# 단계3 : 단계별 모델링


## 0.미션

단계별로 나눠서 모델링을 수행하고자 합니다.  

* 단계1 : 정적(0), 동적(1) 행동 분류 모델 생성
* 단계2 : 세부 동작에 대한 분류모델 생성
    * 단계1 모델에서 0으로 예측 -> 정적 행동 3가지 분류 모델링
    * 단계1 모델에서 1으로 예측 -> 동적 행동 3가지 분류 모델링
* 모델 통합
    * 두 단계 모델을 통합하고, 새로운 데이터에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
* 성능 비교
    * 기본 모델링의 성능과 비교
    * 모든 모델링은 [다양한 알고리즘 + 성능 튜닝]을 수행해야 합니다.


## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용

 <br/>  

* 세부 요구사항
    - data01_train.csv 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.
    - data01_test.csv 를 불러와 'new_data' 이름으로 저장합니다.


In [ ]:
import os
path = '/content/drive/MyDrive/2023.10.25_미니프로젝트5차_데이터 및 실습자료'

data = pd.read_csv(os.path.join(path+'/data01_train.csv'))
data.drop('subject', axis=1, inplace=True)
data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.487737,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163,STANDING
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.237820,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944,LAYING
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,-0.535287,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755,STANDING
3,0.289795,-0.035536,-0.150354,-0.231727,-0.006412,-0.338117,-0.273557,0.014245,-0.347916,0.008288,...,-0.004012,-0.408956,-0.255125,0.612804,0.747381,-0.072944,-0.695819,0.287154,0.111388,WALKING
4,0.394807,0.034098,0.091229,0.088489,-0.106636,-0.388502,-0.010469,-0.109680,-0.346372,0.584131,...,-0.157832,-0.563437,-0.044344,-0.845268,-0.974650,-0.887846,-0.705029,0.264952,0.137758,WALKING_DOWNSTAIRS


In [ ]:
new_data = pd.read_csv(os.path.join(path + '/data01_test.csv'))
new_data.drop('subject', axis=1, inplace=True)
new_data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0.284379,-0.021981,-0.116683,-0.992490,-0.979640,-0.963321,-0.992563,-0.977304,-0.958142,-0.938850,...,-0.509523,-0.850065,-0.018043,0.092304,0.074220,-0.714534,-0.671943,-0.018351,-0.185733,SITTING
1,0.277440,-0.028086,-0.118412,-0.996620,-0.927676,-0.972294,-0.997346,-0.931405,-0.971788,-0.939837,...,-0.210792,-0.613367,-0.022456,-0.155414,0.247498,-0.112257,-0.826816,0.184489,-0.068699,STANDING
2,0.305833,-0.041023,-0.087303,0.006880,0.182800,-0.237984,0.005642,0.028616,-0.236474,0.016311,...,0.579587,0.394388,-0.362616,0.171069,0.576349,-0.688314,-0.743234,0.272186,0.053101,WALKING
3,0.276053,-0.016487,-0.108381,-0.995379,-0.983978,-0.975854,-0.995877,-0.985280,-0.974907,-0.941425,...,-0.566291,-0.841455,0.289548,0.079801,-0.020033,0.291898,-0.639435,-0.111998,-0.123298,SITTING
4,0.271998,0.016904,-0.078856,-0.973468,-0.702462,-0.869450,-0.979810,-0.711601,-0.856807,-0.920760,...,0.447577,0.214219,0.010111,0.114179,-0.830776,-0.325098,-0.840817,0.116237,-0.096615,STANDING


## 2.데이터 전처리

* 세부 요구사항
    - Label 추가 : data 에 Activity_dynamic 를 추가합니다. Activity_dynamic은 과제1에서 is_dynamic과 동일한 값입니다.
    - x와 y1, y2로 분할하시오.
        * y1 : Activity
        * y2 : Activity_dynamic
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [ ]:
data['Activity_dynamic'] = data['Activity'].map({'STANDING' : 0, 'SITTING' : 0, 'LAYING' : 0, 'WALKING' : 1, 'WALKING_UPSTAIRS' : 1, 'WALKING_DOWNSTAIRS' : 1})

In [ ]:
x = data.drop(['Activity_dynamic', 'Activity'], axis=1)
y1 = data['Activity']
y2 = data['Activity_dynamic']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_val, y1_train, y1_val = train_test_split(x, y1, test_size=0.2, random_state=42)
x_train, x_val, y2_train, y2_val = train_test_split(x, y2, test_size=0.2, random_state=42)

## **3.단계별 모델링**

![](https://github.com/DA4BAM/image/blob/main/step%20by%20step.png?raw=true)

In [ ]:
# 변수의 특성 중요도 계산하기
def plot_feature_importance(importance, names, result_only = False, topn = 'all'):
    feature_importance = np.array(importance)
    feature_name = np.array(names)

    data={'feature_name':feature_name,'feature_importance':feature_importance}
    fi_temp = pd.DataFrame(data)

    #변수의 특성 중요도 순으로 정렬하기
    fi_temp.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_temp.reset_index(drop=True, inplace = True)

    if topn == 'all' :
        fi_df = fi_temp.copy()
    else :
        fi_df = fi_temp.iloc[:topn]

    #변수의 특성 중요도 그래프로 그리기
    if result_only == False :
        plt.figure(figsize=(10,20))
        sns.barplot(x='feature_importance', y='feature_name', data = fi_df)

        plt.xlabel('importance')
        plt.ylabel('feature name')
        plt.grid()

    return fi_df

### (1) 단계1 : 정적/동적 행동 분류 모델

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)과 동적 행동(동적 : Walking, Walking-Up, Walking-Down)을 구분하는 모델 생성.
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

#### 1) 알고리즘1 : RandomForestClassifier

1-1 ) 랜덤 포레스트 변수 561개 모두 사용

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_train, y2_train)
y2_pred_rf = model.predict(x_val)

In [ ]:
# 평가하기
print("Accuracy : " , accuracy_score(y2_val, y2_pred_rf))
print('')
print("f1 score : " , f1_score(y2_val, y2_pred_rf, average='macro'))
print('')
print("classification report : " , classification_report(y2_val, y2_pred_rf))

Accuracy :  1.0

f1 score :  1.0

classification report :                precision    recall  f1-score   support

           0       1.00      1.00      1.00       657
           1       1.00      1.00      1.00       520

    accuracy                           1.00      1177
   macro avg       1.00      1.00      1.00      1177
weighted avg       1.00      1.00      1.00      1177



#### 1-2) 변수 개수 줄여가기 (200, 100, 50, 25개)

In [ ]:
result = plot_feature_importance(model.feature_importances_, list(x), True, 'all')

In [ ]:
x200 = list(result.loc[:199, 'feature_name'])
x100 = list(result.loc[:99, 'feature_name'])
x50 = list(result.loc[:49, 'feature_name'])
x25 = list(result.loc[:24, 'feature_name'])

In [ ]:
x_train200 = x_train[x200]
x_val200 = x_val[x200]
x_train100 = x_train[x100]
x_val100 = x_val[x100]
x_train50 = x_train[x50]
x_val50 = x_val[x50]
x_train25 = x_train[x25]
x_val25 = x_val[x25]

In [ ]:
# 200개

from sklearn.ensemble import RandomForestClassifier

model_rf_200 = RandomForestClassifier()
model_rf_200.fit(x_train200, y2_train)
y2_pred_rf_200 = model_rf_200.predict(x_val200)

# 평가하기
print("Accuracy : " , accuracy_score(y2_val, y2_pred_rf_200))
print('')
print("f1 score : " , f1_score(y2_val, y2_pred_rf_200, average='macro'))
print('')
print("classification report : " , classification_report(y2_val, y2_pred_rf_200))

Accuracy :  1.0

f1 score :  1.0

classification report :                precision    recall  f1-score   support

           0       1.00      1.00      1.00       657
           1       1.00      1.00      1.00       520

    accuracy                           1.00      1177
   macro avg       1.00      1.00      1.00      1177
weighted avg       1.00      1.00      1.00      1177



In [ ]:
# 100개

from sklearn.ensemble import RandomForestClassifier

model_rf_100 = RandomForestClassifier()
model_rf_100.fit(x_train100, y2_train)
y2_pred_rf_100 = model_rf_100.predict(x_val100)

# 평가하기
print("Accuracy : " , accuracy_score(y2_val, y2_pred_rf_100))
print('')
print("f1 score : " , f1_score(y2_val, y2_pred_rf_100, average='macro'))
print('')
print("classification report : " , classification_report(y2_val, y2_pred_rf_100))

Accuracy :  1.0

f1 score :  1.0

classification report :                precision    recall  f1-score   support

           0       1.00      1.00      1.00       657
           1       1.00      1.00      1.00       520

    accuracy                           1.00      1177
   macro avg       1.00      1.00      1.00      1177
weighted avg       1.00      1.00      1.00      1177



In [ ]:
# 50개

from sklearn.ensemble import RandomForestClassifier

model_rf_50 = RandomForestClassifier()
model_rf_50.fit(x_train50, y2_train)
y2_pred_rf_50 = model_rf_50.predict(x_val50)

# 평가하기
print("Accuracy : " , accuracy_score(y2_val, y2_pred_rf_50))
print('')
print("f1 score : " , f1_score(y2_val, y2_pred_rf_50, average='macro'))
print('')
print("classification report : " , classification_report(y2_val, y2_pred_rf_50))

Accuracy :  0.9991503823279524

f1 score :  0.9991385399093161

classification report :                precision    recall  f1-score   support

           0       1.00      1.00      1.00       657
           1       1.00      1.00      1.00       520

    accuracy                           1.00      1177
   macro avg       1.00      1.00      1.00      1177
weighted avg       1.00      1.00      1.00      1177



In [ ]:
# 25개

from sklearn.ensemble import RandomForestClassifier

model_rf_25 = RandomForestClassifier()
model_rf_25.fit(x_train25, y2_train)
y2_pred_rf_25 = model_rf_25.predict(x_val25)

# 평가하기
print("Accuracy : " , accuracy_score(y2_val, y2_pred_rf_25))
print('')
print("f1 score : " , f1_score(y2_val, y2_pred_rf_25, average='macro'))
print('')
print("classification report : " , classification_report(y2_val, y2_pred_rf_25))

Accuracy :  0.9983007646559049

f1 score :  0.9982767304437455

classification report :                precision    recall  f1-score   support

           0       1.00      1.00      1.00       657
           1       1.00      1.00      1.00       520

    accuracy                           1.00      1177
   macro avg       1.00      1.00      1.00      1177
weighted avg       1.00      1.00      1.00      1177



#### 2) 알고리즘2 : Hist Gradient Boosting

2-1) 변수 561개 모두 사용 (HGB)

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model_hgb = HistGradientBoostingClassifier()
model_hgb.fit(x_train, y2_train)
y2_pred_hgb = model.predict(x_val)

# 평가하기
print("Accuracy : " , accuracy_score(y2_val, y2_pred_hgb))
print('')
print("f1 score : " , f1_score(y2_val, y2_pred_hgb, average='macro'))
print('')
print("classification report : " , classification_report(y2_val, y2_pred_hgb))

/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


Accuracy :  1.0

f1 score :  1.0

classification report :                precision    recall  f1-score   support

           0       1.00      1.00      1.00       657
           1       1.00      1.00      1.00       520

    accuracy                           1.00      1177
   macro avg       1.00      1.00      1.00      1177
weighted avg       1.00      1.00      1.00      1177



#### 2-2) 변수 개수 줄여가기 (200, 100, 50, 25개)

In [ ]:
# 200개
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model_hgb_200 = HistGradientBoostingClassifier()
model_hgb_200.fit(x_train200, y2_train)
y2_pred_hgb_200 = model_hgb_200.predict(x_val200)

# 평가하기
print("Accuracy : " , accuracy_score(y2_val, y2_pred_hgb_200))
print('')
print("f1 score : " , f1_score(y2_val, y2_pred_hgb_200, average='macro'))
print('')
print("classification report : " , classification_report(y2_val, y2_pred_hgb_200))

Accuracy :  1.0

f1 score :  1.0

classification report :                precision    recall  f1-score   support

           0       1.00      1.00      1.00       657
           1       1.00      1.00      1.00       520

    accuracy                           1.00      1177
   macro avg       1.00      1.00      1.00      1177
weighted avg       1.00      1.00      1.00      1177



In [ ]:
# 100개
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model_hgb_100 = HistGradientBoostingClassifier()
model_hgb_100.fit(x_train100, y2_train)
y2_pred_hgb_100 = model_hgb_100.predict(x_val100)

# 평가하기
print("Accuracy : " , accuracy_score(y2_val, y2_pred_hgb_100))
print('')
print("f1 score : " , f1_score(y2_val, y2_pred_hgb_100, average='macro'))
print('')
print("classification report : " , classification_report(y2_val, y2_pred_hgb_100))

Accuracy :  1.0

f1 score :  1.0

classification report :                precision    recall  f1-score   support

           0       1.00      1.00      1.00       657
           1       1.00      1.00      1.00       520

    accuracy                           1.00      1177
   macro avg       1.00      1.00      1.00      1177
weighted avg       1.00      1.00      1.00      1177



In [ ]:
# 50개
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model_hgb_50 = HistGradientBoostingClassifier()
model_hgb_50.fit(x_train50, y2_train)
y2_pred_hgb_50 = model_hgb_50.predict(x_val50)

# 평가하기
print("Accuracy : " , accuracy_score(y2_val, y2_pred_hgb_50))
print('')
print("f1 score : " , f1_score(y2_val, y2_pred_hgb_50, average='macro'))
print('')
print("classification report : " , classification_report(y2_val, y2_pred_hgb_50))

Accuracy :  1.0

f1 score :  1.0

classification report :                precision    recall  f1-score   support

           0       1.00      1.00      1.00       657
           1       1.00      1.00      1.00       520

    accuracy                           1.00      1177
   macro avg       1.00      1.00      1.00      1177
weighted avg       1.00      1.00      1.00      1177



In [ ]:
# 25개
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model_hgb_25 = HistGradientBoostingClassifier()
model_hgb_25.fit(x_train25, y2_train)
y2_pred_hgb_25 = model_hgb_25.predict(x_val25)

# 평가하기
print("Accuracy : " , accuracy_score(y2_val, y2_pred_hgb_25))
print('')
print("f1 score : " , f1_score(y2_val, y2_pred_hgb_25, average='macro'))
print('')
print("classification report : " , classification_report(y2_val, y2_pred_hgb_25))

Accuracy :  0.9991503823279524

f1 score :  0.9991385399093161

classification report :                precision    recall  f1-score   support

           0       1.00      1.00      1.00       657
           1       1.00      1.00      1.00       520

    accuracy                           1.00      1177
   macro avg       1.00      1.00      1.00      1177
weighted avg       1.00      1.00      1.00      1177



#### 정적/동적 행동 분류 개수별 모델링 결과

In [ ]:
rf_acc = [accuracy_score(y2_val, y2_pred_rf), accuracy_score(y2_val, y2_pred_rf_200), accuracy_score(y2_val, y2_pred_rf_100), accuracy_score(y2_val, y2_pred_rf_50), accuracy_score(y2_val, y2_pred_rf_25)]
hgb_acc = [accuracy_score(y2_val, y2_pred_hgb), accuracy_score(y2_val, y2_pred_hgb_200), accuracy_score(y2_val, y2_pred_hgb_100), accuracy_score(y2_val, y2_pred_hgb_50), accuracy_score(y2_val, y2_pred_hgb_25)]
rf_f1 = [f1_score(y2_val, y2_pred_rf, average='macro'), f1_score(y2_val, y2_pred_rf_200, average='macro'), f1_score(y2_val, y2_pred_rf_100, average='macro'), f1_score(y2_val, y2_pred_rf_50, average='macro'), f1_score(y2_val, y2_pred_rf_25, average='macro')]
hgb_f1 = [f1_score(y2_val, y2_pred_hgb, average='macro'), f1_score(y2_val, y2_pred_hgb_200, average='macro'), f1_score(y2_val, y2_pred_hgb_100, average='macro'), f1_score(y2_val, y2_pred_hgb_50, average='macro'), f1_score(y2_val, y2_pred_hgb_25, average='macro')]
feature_numbers = [561, 200, 100, 50, 25]
static_dynamic = pd.DataFrame({'Feature_numbers' : feature_numbers, 'RandomForest_Accuracy' : rf_acc, 'HistGradientBoosting_Accuracy' : hgb_acc, 'RandomForest_Accuracy_F1_Score' : rf_f1, 'HistGradientBoosting_F1_score' : hgb_f1})
static_dynamic

,Feature_numbers,RandomForest_Accuracy,HistGradientBoosting_Accuracy,RandomForest_Accuracy_F1_Score,HistGradientBoosting_F1_score
0,561,1.000000,1.00000,1.000000,1.000000
1,200,1.000000,1.00000,1.000000,1.000000
2,100,1.000000,1.00000,1.000000,1.000000
3,50,0.999150,1.00000,0.999139,1.000000
4,25,0.998301,0.99915,0.998277,0.999139


### (2) 단계2-1 : 정적 동작 세부 분류

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)인 데이터 추출
    * Laying, Sitting, Standing 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [ ]:
data_static = data[data['Activity'] .isin(['LAYING', 'SITTING', 'STANDING'])]

In [ ]:
x_static = data_static.drop(['Activity'], axis=1)
y_static = data_static['Activity']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_static_train, x_static_val, y_static_train, y_static_val = train_test_split(x_static, y_static, test_size=0.2, random_state=42)

In [ ]:
x_static_train200 = x_static_train[x200]
x_static_val200 = x_static_val[x200]
x_static_train100 = x_static_train[x100]
x_static_val100 = x_static_val[x100]
x_static_train50 = x_static_train[x50]
x_static_val50 = x_static_val[x50]
x_static_train25 = x_static_train[x25]
x_static_val25 = x_static_val[x25]

####  랜덤포레스트

In [ ]:
# 전체

from sklearn.ensemble import RandomForestClassifier

model_static_rf = RandomForestClassifier()
model_static_rf.fit(x_static_train, y_static_train)
y_static_pred_rf = model_static_rf.predict(x_static_val)

# 평가하기
print("Accuracy : " , accuracy_score(y_static_val, y_static_pred_rf))
print('')
print("f1 score : " , f1_score(y_static_val, y_static_pred_rf, average='macro'))
print('')
print("classification report : " , classification_report(y_static_val, y_static_pred_rf))

Accuracy :  0.9706336939721792

f1 score :  0.9702383250508806

classification report :                precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       221
     SITTING       0.95      0.96      0.95       205
    STANDING       0.96      0.95      0.96       221

    accuracy                           0.97       647
   macro avg       0.97      0.97      0.97       647
weighted avg       0.97      0.97      0.97       647



In [ ]:
# 200개

from sklearn.ensemble import RandomForestClassifier

model_static_rf_200 = RandomForestClassifier()
model_static_rf_200.fit(x_static_train200, y_static_train)
y_static_pred_rf_200 = model_static_rf_200.predict(x_static_val200)

# 평가하기
print("Accuracy : " , accuracy_score(y_static_val, y_static_pred_rf_200))
print('')
print("f1 score : " , f1_score(y_static_val, y_static_pred_rf_200, average='macro'))
print('')
print("classification report : " , classification_report(y_static_val, y_static_pred_rf_200))

Accuracy :  0.9721792890262752

f1 score :  0.9718253968253968

classification report :                precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       221
     SITTING       0.93      0.98      0.96       205
    STANDING       0.98      0.94      0.96       221

    accuracy                           0.97       647
   macro avg       0.97      0.97      0.97       647
weighted avg       0.97      0.97      0.97       647



In [ ]:
# 100개

from sklearn.ensemble import RandomForestClassifier

model_static_rf_100 = RandomForestClassifier()
model_static_rf_100.fit(x_static_train100, y_static_train)
y_static_pred_rf_100 = model_static_rf_100.predict(x_static_val100)

# 평가하기
print("Accuracy : " , accuracy_score(y_static_val, y_static_pred_rf_100))
print('')
print("f1 score : " , f1_score(y_static_val, y_static_pred_rf_100, average='macro'))
print('')
print("classification report : " , classification_report(y_static_val, y_static_pred_rf_100))

Accuracy :  0.9675425038639877

f1 score :  0.9671272743307225

classification report :                precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       221
     SITTING       0.93      0.97      0.95       205
    STANDING       0.97      0.93      0.95       221

    accuracy                           0.97       647
   macro avg       0.97      0.97      0.97       647
weighted avg       0.97      0.97      0.97       647



In [ ]:
# 50개

from sklearn.ensemble import RandomForestClassifier

model_static_rf_50 = RandomForestClassifier()
model_static_rf_50.fit(x_static_train50, y_static_train)
y_static_pred_rf_50 = model_static_rf_50.predict(x_static_val50)

# 평가하기
print("Accuracy : " , accuracy_score(y_static_val, y_static_pred_rf_50))
print('')
print("f1 score : " , f1_score(y_static_val, y_static_pred_rf_50, average='macro'))
print('')
print("classification report : " , classification_report(y_static_val, y_static_pred_rf_50))

Accuracy :  0.955177743431221

f1 score :  0.9545963229416468

classification report :                precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       221
     SITTING       0.92      0.95      0.93       205
    STANDING       0.95      0.92      0.93       221

    accuracy                           0.96       647
   macro avg       0.95      0.95      0.95       647
weighted avg       0.96      0.96      0.96       647



In [ ]:
# 25개

from sklearn.ensemble import RandomForestClassifier

model_static_rf_25 = RandomForestClassifier()
model_static_rf_25.fit(x_static_train25, y_static_train)
y_static_pred_rf_25 = model_static_rf_25.predict(x_static_val25)

# 평가하기
print("Accuracy : " , accuracy_score(y_static_val, y_static_pred_rf_25))
print('')
print("f1 score : " , f1_score(y_static_val, y_static_pred_rf_25, average='macro'))
print('')
print("classification report : " , classification_report(y_static_val, y_static_pred_rf_25))

Accuracy :  0.6846986089644513

f1 score :  0.6805536948905099

classification report :                precision    recall  f1-score   support

      LAYING       0.77      0.81      0.79       221
     SITTING       0.59      0.57      0.58       205
    STANDING       0.68      0.67      0.67       221

    accuracy                           0.68       647
   macro avg       0.68      0.68      0.68       647
weighted avg       0.68      0.68      0.68       647



#### HGB

In [ ]:
# 전체
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model_static_hgb = HistGradientBoostingClassifier()
model_static_hgb.fit(x_static_train, y_static_train)
y_static_pred_hgb = model_static_hgb.predict(x_static_val)

# 평가하기
print("Accuracy : " , accuracy_score(y_static_val, y_static_pred_hgb))
print('')
print("f1 score : " , f1_score(y_static_val, y_static_pred_hgb, average='macro'))
print('')
print("classification report : " , classification_report(y_static_val, y_static_pred_hgb))

Accuracy :  0.9845440494590417

f1 score :  0.9843381201332363

classification report :                precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       221
     SITTING       0.97      0.99      0.98       205
    STANDING       0.99      0.97      0.98       221

    accuracy                           0.98       647
   macro avg       0.98      0.98      0.98       647
weighted avg       0.98      0.98      0.98       647



In [ ]:
# 200개
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model_static_hgb_200 = HistGradientBoostingClassifier()
model_static_hgb_200.fit(x_static_train200, y_static_train)
y_static_pred_hgb_200 = model_static_hgb_200.predict(x_static_val200)

# 평가하기
print("Accuracy : " , accuracy_score(y_static_val, y_static_pred_hgb_200))
print('')
print("f1 score : " , f1_score(y_static_val, y_static_pred_hgb_200, average='macro'))
print('')
print("classification report : " , classification_report(y_static_val, y_static_pred_hgb_200))

Accuracy :  0.9799072642967542

f1 score :  0.9796367487190235

classification report :                precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       221
     SITTING       0.96      0.98      0.97       205
    STANDING       0.98      0.96      0.97       221

    accuracy                           0.98       647
   macro avg       0.98      0.98      0.98       647
weighted avg       0.98      0.98      0.98       647



In [ ]:
# 100개
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model_static_hgb_100 = HistGradientBoostingClassifier()
model_static_hgb_100.fit(x_static_train100, y_static_train)
y_static_pred_hgb_100 = model_static_hgb_100.predict(x_static_val100)

# 평가하기
print("Accuracy : " , accuracy_score(y_static_val, y_static_pred_hgb_100))
print('')
print("f1 score : " , f1_score(y_static_val, y_static_pred_hgb_100, average='macro'))
print('')
print("classification report : " , classification_report(y_static_val, y_static_pred_hgb_100))

Accuracy :  0.98145285935085

f1 score :  0.9812103034580099

classification report :                precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       221
     SITTING       0.96      0.99      0.97       205
    STANDING       0.99      0.96      0.97       221

    accuracy                           0.98       647
   macro avg       0.98      0.98      0.98       647
weighted avg       0.98      0.98      0.98       647



In [ ]:
# 50개
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model_static_hgb_50 = HistGradientBoostingClassifier()
model_static_hgb_50.fit(x_static_train50, y_static_train)
y_static_pred_hgb_50 = model_static_hgb_50.predict(x_static_val50)

# 평가하기
print("Accuracy : " , accuracy_score(y_static_val, y_static_pred_hgb_50))
print('')
print("f1 score : " , f1_score(y_static_val, y_static_pred_hgb_50, average='macro'))
print('')
print("classification report : " , classification_report(y_static_val, y_static_pred_hgb_50))

Accuracy :  0.98145285935085

f1 score :  0.9812103034580099

classification report :                precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       221
     SITTING       0.96      0.99      0.97       205
    STANDING       0.99      0.96      0.97       221

    accuracy                           0.98       647
   macro avg       0.98      0.98      0.98       647
weighted avg       0.98      0.98      0.98       647



In [ ]:
# 25개
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model_static_hgb_25 = HistGradientBoostingClassifier()
model_static_hgb_25.fit(x_static_train25, y_static_train)
y_static_pred_hgb_25 = model_static_hgb_25.predict(x_static_val25)

# 평가하기
print("Accuracy : " , accuracy_score(y_static_val, y_static_pred_hgb_25))
print('')
print("f1 score : " , f1_score(y_static_val, y_static_pred_hgb_25, average='macro'))
print('')
print("classification report : " , classification_report(y_static_val, y_static_pred_hgb_25))

Accuracy :  0.7063369397217929

f1 score :  0.7018831760515877

classification report :                precision    recall  f1-score   support

      LAYING       0.81      0.85      0.83       221
     SITTING       0.61      0.59      0.60       205
    STANDING       0.69      0.67      0.68       221

    accuracy                           0.71       647
   macro avg       0.70      0.70      0.70       647
weighted avg       0.70      0.71      0.70       647



#### 정적 분류 성능평가 결과 종합

In [ ]:
rf_acc = [accuracy_score(y_static_val, y_static_pred_rf), accuracy_score(y_static_val, y_static_pred_rf_200), accuracy_score(y_static_val, y_static_pred_rf_100), accuracy_score(y_static_val, y_static_pred_rf_50), accuracy_score(y_static_val, y_static_pred_rf_25)]
hgb_acc = [accuracy_score(y_static_val, y_static_pred_hgb), accuracy_score(y_static_val, y_static_pred_hgb_200), accuracy_score(y_static_val, y_static_pred_hgb_100), accuracy_score(y_static_val, y_static_pred_hgb_50), accuracy_score(y_static_val, y_static_pred_hgb_25)]
rf_f1 = [f1_score(y_static_val, y_static_pred_rf, average='macro'), f1_score(y_static_val, y_static_pred_rf_200, average='macro'), f1_score(y_static_val, y_static_pred_rf_100, average='macro'), f1_score(y_static_val, y_static_pred_rf_50, average='macro'), f1_score(y_static_val, y_static_pred_rf_25, average='macro')]
hgb_f1 = [f1_score(y_static_val, y_static_pred_hgb, average='macro'), f1_score(y_static_val, y_static_pred_hgb_200, average='macro'), f1_score(y_static_val, y_static_pred_hgb_100, average='macro'), f1_score(y_static_val, y_static_pred_hgb_50, average='macro'), f1_score(y_static_val, y_static_pred_hgb_25, average='macro')]
feature_numbers = [561, 200, 100, 50, 25]
static = pd.DataFrame({'Feature_numbers' : feature_numbers, 'RandomForest_Accuracy' : rf_acc, 'HistGradientBoosting_Accuracy' : hgb_acc, 'RandomForest_Accuracy_F1_Score' : rf_f1, 'HistGradientBoosting_F1_score' : hgb_f1})
static

,Feature_numbers,RandomForest_Accuracy,HistGradientBoosting_Accuracy,RandomForest_Accuracy_F1_Score,HistGradientBoosting_F1_score
0,561,0.970634,0.984544,0.970238,0.984338
1,200,0.972179,0.979907,0.971825,0.979637
2,100,0.967543,0.981453,0.967127,0.981210
3,50,0.955178,0.981453,0.954596,0.981210
4,25,0.684699,0.706337,0.680554,0.701883


### (3) 단계2-2 : 동적 동작 세부 분류

* 세부 요구사항
    * 동동적 행동(Walking, Walking Upstairs, Walking Downstairs)인 데이터 추출
    * Walking, Walking Upstairs, Walking Downstairs 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [ ]:
data_dyn = data[data['Activity'] .isin(['WALKING', 'WALKING UPSTAIRS', 'WALKING DOWNSTAIRS'])]

In [ ]:
x_dyn = data_dyn.drop(['Activity'], axis=1)
y_dyn = data_dyn['Activity']

In [ ]:
from sklearn.model_selection import train_test_split

x_dyn_train, x_dyn_val, y_dyn_train, y_dyn_val = train_test_split(x_dyn, y_dyn, test_size=0.2, random_state=42)

In [ ]:
x_dyn_train200 = x_dyn_train[x200]
x_dyn_val200 = x_dyn_val[x200]
x_dyn_train100 = x_dyn_train[x100]
x_dyn_val100 = x_dyn_val[x100]
x_dyn_train50 = x_dyn_train[x50]
x_dyn_val50 = x_dyn_val[x50]
x_dyn_train25 = x_dyn_train[x25]
x_dyn_val25 = x_dyn_val[x25]

#### 랜덤 포레스트

In [ ]:
# 전체

from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(x_dyn_train, y_dyn_train)
y_dyn_pred_rf = model_rf.predict(x_dyn_val)

# 평가하기
print("Accuracy : " , accuracy_score(y_dyn_val, y_dyn_pred_rf))
print('')
print("f1 score : " , f1_score(y_dyn_val, y_dyn_pred_rf, average='macro'))
print('')
print("classification report : " , classification_report(y_dyn_val, y_dyn_pred_rf))

Accuracy :  1.0

f1 score :  1.0

classification report :                precision    recall  f1-score   support

     WALKING       1.00      1.00      1.00       200

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [ ]:
# 200

from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(x_dyn_train200, y_dyn_train)
y_dyn_pred_rf_200 = model_rf.predict(x_dyn_val200)

# 평가하기
print("Accuracy : " , accuracy_score(y_dyn_val, y_dyn_pred_rf_200))
print('')
print("f1 score : " , f1_score(y_dyn_val, y_dyn_pred_rf_200, average='macro'))
print('')
print("classification report : " , classification_report(y_dyn_val, y_dyn_pred_rf_200))

Accuracy :  1.0

f1 score :  1.0

classification report :                precision    recall  f1-score   support

     WALKING       1.00      1.00      1.00       200

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [ ]:
# 100

from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(x_dyn_train100, y_dyn_train)
y_dyn_pred_rf_100 = model_rf.predict(x_dyn_val100)

# 평가하기
print("Accuracy : " , accuracy_score(y_dyn_val, y_dyn_pred_rf_100))
print('')
print("f1 score : " , f1_score(y_dyn_val, y_dyn_pred_rf_100, average='macro'))
print('')
print("classification report : " , classification_report(y_dyn_val, y_dyn_pred_rf_100))

Accuracy :  1.0

f1 score :  1.0

classification report :                precision    recall  f1-score   support

     WALKING       1.00      1.00      1.00       200

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [ ]:
# 50

from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(x_dyn_train50, y_dyn_train)
y_dyn_pred_rf_50 = model_rf.predict(x_dyn_val50)

# 평가하기
print("Accuracy : " , accuracy_score(y_dyn_val, y_dyn_pred_rf_50))
print('')
print("f1 score : " , f1_score(y_dyn_val, y_dyn_pred_rf_50, average='macro'))
print('')
print("classification report : " , classification_report(y_dyn_val, y_dyn_pred_rf_50))

Accuracy :  1.0

f1 score :  1.0

classification report :                precision    recall  f1-score   support

     WALKING       1.00      1.00      1.00       200

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [ ]:
# 25

from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(x_dyn_train25, y_dyn_train)
y_dyn_pred_rf_25 = model_rf.predict(x_dyn_val25)

# 평가하기
print("Accuracy : " , accuracy_score(y_dyn_val, y_dyn_pred_rf_25))
print('')
print("f1 score : " , f1_score(y_dyn_val, y_dyn_pred_rf_25, average='macro'))
print('')
print("classification report : " , classification_report(y_dyn_val, y_dyn_pred_rf_25))

Accuracy :  1.0

f1 score :  1.0

classification report :                precision    recall  f1-score   support

     WALKING       1.00      1.00      1.00       200

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



#### HGB

In [ ]:
# 전체
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model_hgb = HistGradientBoostingClassifier()

model_hgb.fit(x_dyn_train, y_dyn_train)
y_dyn_pred_hgb = model_hgb.predict(x_dyn_val)

# 평가하기
print("Accuracy : " , accuracy_score(y_dyn_val, y_dyn_pred_hgb))
print('')
print("f1 score : " , f1_score(y_dyn_val, y_dyn_pred_hgb, average='macro'))
print('')
print("classification report : " , classification_report(y_dyn_val, y_dyn_pred_hgb))

Accuracy :  1.0

f1 score :  1.0

classification report :                precision    recall  f1-score   support

     WALKING       1.00      1.00      1.00       200

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [ ]:
# 200
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model_hgb = HistGradientBoostingClassifier()

model_hgb.fit(x_dyn_train200, y_dyn_train)
y_dyn_pred_hgb_200 = model_hgb.predict(x_dyn_val200)

# 평가하기
print("Accuracy : " , accuracy_score(y_dyn_val, y_dyn_pred_hgb_200))
print('')
print("f1 score : " , f1_score(y_dyn_val, y_dyn_pred_hgb_200, average='macro'))
print('')
print("classification report : " , classification_report(y_dyn_val, y_dyn_pred_hgb_200))

Accuracy :  1.0

f1 score :  1.0

classification report :                precision    recall  f1-score   support

     WALKING       1.00      1.00      1.00       200

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [ ]:
# 100
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model_hgb = HistGradientBoostingClassifier()

model_hgb.fit(x_dyn_train100, y_dyn_train)
y_dyn_pred_hgb_100 = model_hgb.predict(x_dyn_val100)

# 평가하기
print("Accuracy : " , accuracy_score(y_dyn_val, y_dyn_pred_hgb_100))
print('')
print("f1 score : " , f1_score(y_dyn_val, y_dyn_pred_hgb_100, average='macro'))
print('')
print("classification report : " , classification_report(y_dyn_val, y_dyn_pred_hgb_100))

Accuracy :  1.0

f1 score :  1.0

classification report :                precision    recall  f1-score   support

     WALKING       1.00      1.00      1.00       200

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [ ]:
# 50
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model_hgb = HistGradientBoostingClassifier()

model_hgb.fit(x_dyn_train50, y_dyn_train)
y_dyn_pred_hgb_50 = model_hgb.predict(x_dyn_val50)

# 평가하기
print("Accuracy : " , accuracy_score(y_dyn_val, y_dyn_pred_hgb_50))
print('')
print("f1 score : " , f1_score(y_dyn_val, y_dyn_pred_hgb_50, average='macro'))
print('')
print("classification report : " , classification_report(y_dyn_val, y_dyn_pred_hgb_50))

Accuracy :  1.0

f1 score :  1.0

classification report :                precision    recall  f1-score   support

     WALKING       1.00      1.00      1.00       200

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [ ]:
# 25
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model_hgb = HistGradientBoostingClassifier()

model_hgb.fit(x_dyn_train25, y_dyn_train)
y_dyn_pred_hgb_25 = model_hgb.predict(x_dyn_val25)

# 평가하기
print("Accuracy : " , accuracy_score(y_dyn_val, y_dyn_pred_hgb_25))
print('')
print("f1 score : " , f1_score(y_dyn_val, y_dyn_pred_hgb_25, average='macro'))
print('')
print("classification report : " , classification_report(y_dyn_val, y_dyn_pred_hgb_25))

Accuracy :  1.0

f1 score :  1.0

classification report :                precision    recall  f1-score   support

     WALKING       1.00      1.00      1.00       200

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



#### 종합

In [ ]:
rf_acc = [accuracy_score(y_dyn_val, y_dyn_pred_rf), accuracy_score(y_dyn_val, y_dyn_pred_rf_200), accuracy_score(y_dyn_val, y_dyn_pred_rf_100), accuracy_score(y_dyn_val, y_dyn_pred_rf_50), accuracy_score(y_dyn_val, y_dyn_pred_rf_25)]
hgb_acc = [accuracy_score(y_dyn_val, y_dyn_pred_hgb), accuracy_score(y_dyn_val, y_dyn_pred_hgb_200), accuracy_score(y_dyn_val, y_dyn_pred_hgb_100), accuracy_score(y_dyn_val, y_dyn_pred_hgb_50), accuracy_score(y_dyn_val, y_dyn_pred_hgb_25)]
rf_f1 = [f1_score(y_dyn_val, y_dyn_pred_rf, average='macro'), f1_score(y_dyn_val, y_dyn_pred_rf_200, average='macro'), f1_score(y_dyn_val, y_dyn_pred_rf_100, average='macro'), f1_score(y_dyn_val, y_dyn_pred_rf_50, average='macro'), f1_score(y_dyn_val, y_dyn_pred_rf_25, average='macro')]
hgb_f1 = [f1_score(y_dyn_val, y_dyn_pred_hgb, average='macro'), f1_score(y_dyn_val, y_dyn_pred_hgb_200, average='macro'), f1_score(y_dyn_val, y_dyn_pred_hgb_100, average='macro'), f1_score(y_dyn_val, y_dyn_pred_hgb_50, average='macro'), f1_score(y_dyn_val, y_dyn_pred_hgb_25, average='macro')]
feature_numbers = [561, 200, 100, 50, 25]
dynamic = pd.DataFrame({'Feature_numbers' : feature_numbers, 'RandomForest_Accuracy' : rf_acc, 'HistGradientBoosting_Accuracy' : hgb_acc, 'RandomForest_Accuracy_F1_Score' : rf_f1, 'HistGradientBoosting_F1_score' : hgb_f1})
dynamic

,Feature_numbers,RandomForest_Accuracy,HistGradientBoosting_Accuracy,RandomForest_Accuracy_F1_Score,HistGradientBoosting_F1_score
0,561,1.0,1.0,1.0,1.0
1,200,1.0,1.0,1.0,1.0
2,100,1.0,1.0,1.0,1.0
3,50,1.0,1.0,1.0,1.0
4,25,1.0,1.0,1.0,1.0


### (4) 분류 모델 합치기


* 세부 요구사항
    * 두 단계 모델을 통합하고, 새로운 데이터(test)에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
    * 데이터 파이프라인 구축 : test데이터가 로딩되어 전처리 과정을 거치고, 예측 및 성능 평가 수행

![](https://github.com/DA4BAM/image/blob/main/pipeline%20function.png?raw=true)

#### 1) 함수 만들기

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/2023.10.25_미니프로젝트5차_데이터 및 실습자료/data01_train.csv')
data.drop('subject', axis=1, inplace=True)
data['Activity_dynamic'] = data['Activity'].map({'STANDING' : 0, 'SITTING' : 0, 'LAYING' : 0, 'WALKING' : 1, 'WALKING_UPSTAIRS' : 1, 'WALKING_DOWNSTAIRS' : 1})
data['Activity'] = data['Activity'].map({'STANDING' : 0, 'SITTING' : 1, 'LAYING' : 2, 'WALKING' : 3, 'WALKING_UPSTAIRS' : 4, 'WALKING_DOWNSTAIRS' : 5})
new_data = pd.read_csv('/content/drive/MyDrive/2023.10.25_미니프로젝트5차_데이터 및 실습자료/data01_test.csv')
new_data.drop('subject', axis=1, inplace=True)
new_data['Activity_dynamic'] = new_data['Activity'].map({'STANDING' : 0, 'SITTING' : 0, 'LAYING' : 0, 'WALKING' : 1, 'WALKING_UPSTAIRS' : 1, 'WALKING_DOWNSTAIRS' : 1})
new_data['Activity'] = new_data['Activity'].map({'STANDING' : 0, 'SITTING' : 1, 'LAYING' : 2, 'WALKING' : 3, 'WALKING_UPSTAIRS' : 4, 'WALKING_DOWNSTAIRS' : 5})

In [ ]:
def modeling(train, test, top):
    x_train = train.drop(['Activity', 'Activity_dynamic'], axis=1)
    x_train = x_train[list(result.loc[:top-1, 'feature_name'])]
    y2_train = train['Activity_dynamic']
    x_test = test.drop(['Activity', 'Activity_dynamic'], axis=1)
    x_test = x_test[list(result.loc[:top-1, 'feature_name'])]
    y2_test = test['Activity_dynamic']

    model1 = RandomForestClassifier()
    model1.fit(x_train, y2_train)
    y_pred_1 = model1.predict(x_test)

    y_1s = []
    y_0s = []
    for i in range(len(y_pred_1)):
        if y_pred_1[i] == 1:
            y_1s.append(i)
        else:
            y_0s.append(i)

    x_train_static = train[train['Activity_dynamic'] == 0][list(result.loc[:top-1, 'feature_name'])]
    y1_train_static = train[train['Activity_dynamic'] == 0]['Activity']
    x_test_static = x_test.loc[y_0s]
    y1_test_static = test.loc[y_0s]['Activity']
    x_train_dyn = train[train['Activity_dynamic'] == 1][list(result.loc[:top-1, 'feature_name'])]
    y1_train_dyn = train[train['Activity_dynamic'] == 1]['Activity']
    x_test_dyn = x_test.loc[y_1s]
    y1_test_dyn = test.loc[y_1s]['Activity']

    model2_s = HistGradientBoostingClassifier()
    model2_s.fit(x_train_static, y1_train_static)
    y_pred_2_s = model2_s.predict(x_test_static)

    model2_d = HistGradientBoostingClassifier()
    model2_d.fit(x_train_dyn, y1_train_dyn)
    y_pred_2_d = model2_d.predict(x_test_dyn)

    y_pred_final = y_pred_2_s.tolist() + y_pred_2_d.tolist()
    y_final = y1_test_static.tolist() + y1_test_dyn.tolist()

    print("Accuracy : " , accuracy_score(y_pred_final, y_final))
    print("F1-score : " , f1_score(y_pred_final, y_final, average='macro'))
    print("Classification Report : " , classification_report(y_pred_final, y_final))
    y_pred_2_s_map = list(pd.Series(y_pred_2_s.tolist()).map({0:'STANDING', 1:'SITTING', 2:'LAYING'}))
    y_pred_2_d_map = list(pd.Series(y_pred_2_d.tolist()).map({3:'WALKING', 4:'WALKING_UPSTAIRS', 5:'WALKING_DOWNSTAIRS'}))

    y_final_mapped = []
    j1 = 0
    j0 = 0
    for i in range(len(y2_test)):
        if i in y_1s:
            y_final_mapped.append(y_pred_2_d_map[j1])
            j1 += 1
        else:
            y_final_mapped.append(y_pred_2_s_map[j0])
            j0 += 1

    return y_final_mapped

In [ ]:
y_final_561 = modeling(data, new_data, 561)

Accuracy :  0.991162474507138
F1-score :  0.9915326525522924
Classification Report :                precision    recall  f1-score   support

           0       0.99      0.98      0.99       289
           1       0.98      0.98      0.98       254
           2       0.99      1.00      1.00       290
           3       0.99      1.00      0.99       227
           4       1.00      0.99      1.00       217
           5       0.99      1.00      1.00       194

    accuracy                           0.99      1471
   macro avg       0.99      0.99      0.99      1471
weighted avg       0.99      0.99      0.99      1471



In [ ]:
y_final_200 = modeling(data, new_data, 200)

Accuracy :  0.9843643779741672
F1-score :  0.9852267648246936
Classification Report :                precision    recall  f1-score   support

           0       0.97      0.97      0.97       288
           1       0.96      0.96      0.96       255
           2       0.99      1.00      1.00       290
           3       0.99      1.00      0.99       226
           4       1.00      0.99      0.99       218
           5       0.99      1.00      1.00       194

    accuracy                           0.98      1471
   macro avg       0.99      0.99      0.99      1471
weighted avg       0.98      0.98      0.98      1471



In [ ]:
y_final_100 = modeling(data, new_data, 100)

Accuracy :  0.9891230455472467
F1-score :  0.9899391065791278
Classification Report :                precision    recall  f1-score   support

           0       0.99      0.97      0.98       291
           1       0.96      0.98      0.97       251
           2       0.99      1.00      0.99       291
           3       1.00      1.00      1.00       227
           4       1.00      1.00      1.00       215
           5       1.00      0.99      1.00       196

    accuracy                           0.99      1471
   macro avg       0.99      0.99      0.99      1471
weighted avg       0.99      0.99      0.99      1471



In [ ]:
y_final_25 = modeling(data, new_data, 25)

Accuracy :  0.8082936777702243
F1-score :  0.8217024959731482
Classification Report :                precision    recall  f1-score   support

           0       0.70      0.66      0.68       303
           1       0.59      0.60      0.60       249
           2       0.81      0.84      0.83       282
           3       0.98      0.94      0.96       236
           4       0.90      0.94      0.92       207
           5       0.94      0.95      0.95       194

    accuracy                           0.81      1471
   macro avg       0.82      0.82      0.82      1471
weighted avg       0.81      0.81      0.81      1471



In [ ]:
y_final = modeling(data, new_data, 50)

Accuracy :  0.9809653297076818
F1-score :  0.9819231628030182
Classification Report :                precision    recall  f1-score   support

           0       0.95      0.98      0.96       281
           1       0.97      0.94      0.96       261
           2       0.99      1.00      0.99       291
           3       0.99      1.00      0.99       226
           4       1.00      0.99      0.99       216
           5       0.99      0.99      0.99       196

    accuracy                           0.98      1471
   macro avg       0.98      0.98      0.98      1471
weighted avg       0.98      0.98      0.98      1471



In [ ]:
new_data_1 = pd.read_csv('/content/drive/MyDrive/2023.10.25_미니프로젝트5차_데이터 및 실습자료/data01_test.csv')
new_data_1['Activity']

0                  SITTING
1                 STANDING
2                  WALKING
3                  SITTING
4                 STANDING
               ...        
1466               SITTING
1467              STANDING
1468    WALKING_DOWNSTAIRS
1469                LAYING
1470    WALKING_DOWNSTAIRS
Name: Activity, Length: 1471, dtype: object

In [ ]:
print("정답 : ", list(new_data_1['Activity']))
print("예측값 : ", y_final)
print(" ")
print("정확도 : ", accuracy_score(y_final, new_data_1['Activity']))
print("f1-score : ", f1_score(y_final, new_data_1['Activity'], average='macro'))
print(" ")
print("Classification Report")
print(classification_report(y_final, new_data_1['Activity']))

정답 :  ['SITTING', 'STANDING', 'WALKING', 'SITTING', 'STANDING', 'WALKING', 'STANDING', 'STANDING', 'SITTING', 'WALKING_UPSTAIRS', 'STANDING', 'STANDING', 'LAYING', 'LAYING', 'SITTING', 'LAYING', 'WALKING_UPSTAIRS', 'WALKING', 'WALKING_DOWNSTAIRS', 'WALKING', 'WALKING_UPSTAIRS', 'LAYING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'LAYING', 'SITTING', 'STANDING', 'LAYING', 'SITTING', 'LAYING', 'WALKING', 'STANDING', 'STANDING', 'WALKING_DOWNSTAIRS', 'SITTING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS', 'STANDING', 'WALKING_DOWNSTAIRS', 'WALKING', 'WALKING_UPSTAIRS', 'STANDING', 'SITTING', 'LAYING', 'LAYING', 'LAYING', 'WALKING_UPSTAIRS', 'LAYING', 'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'WALKING', 'LAYING', 'SITTING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'WALKING', 'STANDING', 'SITTING', 'STANDING', 'WALKING', 'LAYING', 'SITTING', 'LAYING', 'WALKING', 'WALKING', 'WALKING', 'SITTING', 'STANDING', 'LA